In [2]:
import pandas as pd, numpy as np

#, csv, requests, datetime, time, zipfile

In [ ]:
path = 'Neighborhoods/' # select folder with data in it

#specify datatypes so FIPS numbers, etc. read in properly
dtypes = {'Geo_STATE': 'str', 'Geo_FIPS': 'str', 'Geo_COUNTY': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

# load acs data
colnames = ['Geo_STATE','Geo_FIPS','ACS16_5yr_B25003003','ACS16_5yr_B25003001','ACS16_5yr_B03002012','ACS16_5yr_B03002003',
            'ACS16_5yr_B03002004','ACS16_5yr_B03002001','ACS16_5yr_B03002012','ACS16_5yr_C17002001','ACS16_5yr_B25004001',
            'ACS16_5yr_C17002008','ACS16_5yr_C17002001','ACS16_5yr_C17002008','ACS16_5yr_C17002001','ACS16_5yr_B25024002',
            'ACS16_5yr_B25024003','ACS16_5yr_B25024004','ACS16_5yr_B25024005','ACS16_5yr_B25024006','ACS16_5yr_B25024007',
            'ACS16_5yr_B25024008','ACS16_5yr_B25024009','ACS16_5yr_B25024001','ACS16_5yr_B25034002','ACS16_5yr_B25034003',
            'ACS16_5yr_B25034004','ACS16_5yr_B25034010','ACS16_5yr_B25034011','ACS16_5yr_B25034001','ACS16_5yr_B15003022',
            'ACS16_5yr_B15003023','ACS16_5yr_B15003024','ACS16_5yr_B15003025','ACS16_5yr_B15003001','ACS16_5yr_B11005001',
            'ACS16_5yr_B11005002','ACS16_5yr_B03002005','ACS16_5yr_B03002006','ACS16_5yr_B03002007','Geo_COUNTY',
            'Density_OccHU_LandSqmi_2016','Geo_landsqmi','PopWtCent_longitude','PopWtCent_latitude',
            'ACS16_5yr_B25001001']

acs_data = pd.read_csv(path + 'ACS_Neighborhood_Raw.csv', usecols=(colnames),delimiter=',', dtype=dtypes)

acs_data.head()

In [ ]:
#this file contains variables like rent, house values, unemployment
add_acs = pd.read_csv(path + 'Additional_ACS.csv', usecols=('Geo_FIPS', 'ACS16_5yr_B25064001', 'ACS16_5yr_B25077001',
                                                            'ACS16_5yr_B23025005','ACS16_5yr_B23025003'), 
                      delimiter=',', dtype=dtypes)
add_acs['FIPS_County'] = add_acs['Geo_FIPS'].str[:5]

#this file contains a cross tab between poverty status and race
colnames3 = ['Geo_FIPS', 'ACS16_5yr_B17020H001', 'ACS16_5yr_B17020H002', 'ACS16_5yr_B17020B001', 'ACS16_5yr_B17020B002', 
             'ACS16_5yr_B17020D001', 'ACS16_5yr_B17020D002', 'ACS16_5yr_B17020E001', 'ACS16_5yr_B17020E002',
             'ACS16_5yr_B17020I001', 'ACS16_5yr_B17020I002']
pov_race = pd.read_csv(path + 'Poverty_race.csv', usecols=(colnames3), delimiter=',', dtype=dtypes)

#this file contains household income by gross rent
colnames2 = ['Geo_FIPS','ACS16_5yr_B25074001','ACS16_5yr_B25074006','ACS16_5yr_B25074007','ACS16_5yr_B25074008',
            'ACS16_5yr_B25074009','ACS16_5yr_B25074015','ACS16_5yr_B25074016','ACS16_5yr_B25074017','ACS16_5yr_B25074018',
            'ACS16_5yr_B25074024','ACS16_5yr_B25074025','ACS16_5yr_B25074026','ACS16_5yr_B25074027','ACS16_5yr_B25074033',
            'ACS16_5yr_B25074034','ACS16_5yr_B25074035','ACS16_5yr_B25074036']
inc_rent = pd.read_csv(path + 'rent_income.csv', usecols=(colnames2), delimiter=',', dtype=dtypes)

#this file contains county data for unemployment rate, median rent, median house values
#Need to change variable names so they are differentiated from the census tract level variables
#colnames4 = ['Geo_FIPS','ACS16_5yr_B23025003','ACS16_5yr_B23025005','B25064001','B25077001']
colnames4 = [0, 57, 59, 69, 71]
newnames = ['FIPS_County','county_labor','county_unemp','county_med_rent','county_med_value']
county = pd.read_csv(path + 'county_data.csv', usecols=(colnames4), header=0, names=(newnames), delimiter=',', dtype=dtypes)

#this file contains jobs near tracts
employ = 'Employment/'
jobs = pd.read_csv(employ + 'tract nearby jobs master.csv', delimiter=',', dtype=dtypes)

county.head()

In [ ]:
#merge together census tract datasets
merge1 = pd.merge(add_acs, pov_race, on="Geo_FIPS", how="outer")
merge2 = pd.merge(merge1, inc_rent, on="Geo_FIPS", how="outer")
merge3 = pd.merge(merge2, jobs, left_on = 'Geo_FIPS', right_on = 'h_tract', how="left")
merge4 = pd.merge(merge3, county, on = 'FIPS_County', how='left')

merge4.head()

In [ ]:
#left merge on this last one since Mark's dataset contains the entire US but we want to keep only CA tracts
master = pd.merge(merge4, acs_data, on='Geo_FIPS',how="left")

master.head()

In [ ]:
out = pd.DataFrame(master['Geo_FIPS']).join(master.loc[:,'alljobs':'county_med_value'])

out['renters'] = master['ACS16_5yr_B25003003']
out['occ_HU'] = master['ACS16_5yr_B25003001']

out['hispanic'] = master['ACS16_5yr_B03002012']
out['black'] = master['ACS16_5yr_B03002004']
out['white'] = master['ACS16_5yr_B03002003']
out['asian'] = master['ACS16_5yr_B03002006']
out['nhpi'] = master['ACS16_5yr_B03002007']
out['total_pop'] = master['ACS16_5yr_B03002001']

out['below200pov'] = master['ACS16_5yr_C17002001'] - master['ACS16_5yr_C17002008']
out['total_pov_status'] = master['ACS16_5yr_C17002001']

out['rent_occ_HU'] = master['ACS16_5yr_B25074001']
out['low_inc_rent_burden'] = (master['ACS16_5yr_B25074006'] + master['ACS16_5yr_B25074007'] + master['ACS16_5yr_B25074008'] + master['ACS16_5yr_B25074009'] + 
                              master['ACS16_5yr_B25074015'] + master['ACS16_5yr_B25074016'] + master['ACS16_5yr_B25074017'] + master['ACS16_5yr_B25074018'] + 
                              master['ACS16_5yr_B25074024'] + master['ACS16_5yr_B25074025'] + master['ACS16_5yr_B25074026'] + master['ACS16_5yr_B25074027'] + 
                              master['ACS16_5yr_B25074033'] + master['ACS16_5yr_B25074034'] + master['ACS16_5yr_B25074035'] + master['ACS16_5yr_B25074036'])

out['white_pov_tot'] = master['ACS16_5yr_B17020H001'] #NH white total
out['white_pov'] = master['ACS16_5yr_B17020H002'] #NH white poverty
out['black_pov_tot'] = master['ACS16_5yr_B17020B001'] #black total
out['black_pov'] = master['ACS16_5yr_B17020B002'] #black pov
out['asian_pov_tot'] = master['ACS16_5yr_B17020D001'] #asian total
out['asian_pov'] = master['ACS16_5yr_B17020D002'] #asian pov
out['nhpi_pov_tot'] = master['ACS16_5yr_B17020E001'] #NHPI total
out['nhpi_pov'] = master['ACS16_5yr_B17020E002'] #NHPI pov
out['hispanic_pov_tot'] = master['ACS16_5yr_B17020I001'] #hispanic total
out['hispanic_pov'] = master['ACS16_5yr_B17020I002'] #hispanic pov

out['sfdetach'] = master['ACS16_5yr_B25024002']
out['smallmf'] = master['ACS16_5yr_B25024004'] + master['ACS16_5yr_B25024005'] #2 units and 3-4 units
out['medmf'] = master['ACS16_5yr_B25024006'] + master['ACS16_5yr_B25024007'] #5-9 units and 10-19 units
out['bigmf'] = master['ACS16_5yr_B25024008'] + master['ACS16_5yr_B25024009'] #20-49 units and 50+ units
out['total_hu'] = master['ACS16_5yr_B25024001']
out['vacant_hu'] = master['ACS16_5yr_B25004001']
out['total_hu2'] = master['ACS16_5yr_B25001001']

out['since2000'] = master['ACS16_5yr_B25034002'] + master['ACS16_5yr_B25034003'] + master['ACS16_5yr_B25034004'] #2014 or later, 2010-2013, 2000-2009
out['before1950'] = master['ACS16_5yr_B25034010'] + master['ACS16_5yr_B25034011'] #1940-1949, 1939 or earlier
out['total_structure'] = master['ACS16_5yr_B25034001']

out['bach_degree'] = master['ACS16_5yr_B15003022'] + master['ACS16_5yr_B15003023'] + master['ACS16_5yr_B15003024'] + master['ACS16_5yr_B15003025']
out['pop_over_25'] = master['ACS16_5yr_B15003001']

out['hh_kids'] = master['ACS16_5yr_B11005001']
out['total_hh'] = master['ACS16_5yr_B11005002']

out['med_rent'] = master['ACS16_5yr_B25064001']
out['med_housevalue'] = master['ACS16_5yr_B25077001']

out['unemp_civ'] = master['ACS16_5yr_B23025005']
out['civ_labor_force'] = master['ACS16_5yr_B23025003']

out.head()


In [ ]:
out.to_csv(path + 'tract_data.csv')